# Probability distribution animation generator

To explain how an algorithm works it is sometimes usefull to see how the probability of getting some results evolve.
Quantanium comes with a function to generate an animation showing the evolution of the probability distribution of each result.

### Import the animation generator

In [1]:
from quantanium import *
from mimiqcircuits import *
from quantanium.visualization import generate_animation


import numpy as np
# the following module is only used to display videos in the notebook
from IPython.display import Video

### GHZ Animated

The generate_animation can be used to generate an animation out of any basic circuit built with the framework.

Let's start by writing a function to generate a GHZ state.

In [2]:
def ghz_circ(qubits):
    ghz = Circuit()
    ghz.push(GateH(), 0)
    ghz.push(GateCX(), 0, range(1, qubits))
    return ghz

To generate the animation you simply call the generate_animation function and give it a circuit.

In [3]:
qubits = 4
video_name = f"videos/ghz{qubits}_animation.mp4"

# Generate the circuit
circuit = ghz_circ(qubits)

# Generate the Video
generate_animation(circuit, file_name=video_name)
Video(video_name)

Generating frames for instruction: 4/4
Video saved in videos/ghz4_animation.mp4


In the example above you can see all the states on the y axis and the probability to callapse on any y state if all qubits are measured.

If you want to use a circuit of more than 5 qubits the animation will automatically group together all subsequent states that have the same value.

For Instance GHZ 15 generates the following:

In [4]:
qubits = 15
video_name = f"videos/ghz{qubits}_animation.mp4"
circuit = ghz_circ(qubits)
generate_animation(circuit, file_name=video_name)
Video(video_name)

Generating frames for instruction: 15/15
Video saved in videos/ghz15_animation.mp4


In the example above on the first frame of the video all the states from 000...01 to 111....1 have the same probability of 0, therefore, they are being represented with one singular bar on the graph, meaning that all the bars have the same probability of 0 and that the sum of all the bars is equal to 0

## Let's see a more complex example with Grover Animated

Grover is a much more complex algorithm than GHZ and needs to apply many more instructions to hold results, it is not reasonable to generate an animation showing every instruction making evolve the probability distribution.

In the case of grover you may want to show the evolution of the probability distribution after each grover iteration instead of each instruction.

Here is an example on how to generate such an animation:

### Setting up the animation arguments

We start by generating two empty list:

animation_step will be used to indicate after what instruction to generate  a new step in the animation. For example if I have a circuit of 200 instructions and I give the list [1, 50, 120] the animation will only generates the evolution between the 1st and 50th instructions then between the 50th and 120th instructions

animation_titles must be of the same length as animation_step, it will be used to display a title above the graph between each step

In [5]:
animation_steps = []
animation_titles = []

Next we can define Grover's algorithm. for more details on this grover implementation please take a look at [this notebook](https://github.com/qperfect-io/QleoNotebooks/blob/main/usecases/cybersecurity/grover.ipynb).

Note how we update the animation_step and animation_titles after each grover iteration

In [6]:
def grover(iterations, oracle_circ):
    """
    MIMIQ circuit for Grover's algorithm.
    
    Args:
    iterations (int): Number of Grover iterations.
    oracle_circ (Circuit): Oracle circuit.
    
    Returns:
    Circuit: The Grover's algorithm quantum circuit.
    """
    nq = oracle_circ.num_qubits()
    circ = Circuit()
    
    # Create the uniform superposition state
    circ.push(GateH(), range(nq))

    #### For stepping animation
    global animation_steps
    global animation_titles

    animation_steps.append(len(circ) - 1)  # !!!!!!!! First step of the animation after applying the Hadamards
    animation_titles.append("Initial Uniform state")
    
    for i in range(iterations):
            
        # Apply the oracle operator
        circ.append(oracle_circ)
        
        # Apply the diffusion operator
        circ.push(Diffusion(nq), *range(nq))

        ##### For stepping animation
        animation_steps.append(len(circ) - 1) # !!!!!!!!!!!!!!! New step for the animation after each grover iteration
        animation_titles.append(f"Grover iteration {i + 1}")
    return circ

We define a very simple oracle for Grover

In [7]:
def oracle(bitstring):
    """
    Create an oracle circuit that flips the phase of a desired bitstring.
    
    Args:
    bitstring: Target bitstring to mark.
    
    Returns:
    Circuit: Oracle circuit that marks the target state.
    """
    nq = len(bitstring)
    circ = Circuit()

    # Apply X gates where bits are 0
    for i, b in enumerate(bitstring):
        if not b:
            circ.push(GateX(), i)

    # Multi-controlled Z gate
    circ.push(control(nq-1, GateZ()), *range(nq))

    # Uncompute X gates
    for i, b in enumerate(bitstring):
        if not b:
            circ.push(GateX(),i)
            
    return circ

We are now going to generate the animation.

For this example of grover we are only interested in one single state defined in the code block just below, for this reason we are going to ask the simulation to focus on this state specifically.

We are also going to give the animation_steps and titles argument to only show the grover iterations steps and not every sngle instruction in the circuit

In [8]:
# define parameters
targetstr = BitString([1, 0, 1, 0, 1, 0])
maxiter=7
n = len(targetstr)


# Just to be sure let's reinitialize the animation's arguments before calling the grover function
animation_steps = []
animation_titles = []


# generate circuits and fills animation_steps/titles
circuit = grover(maxiter, oracle(targetstr))

# make sure this is the same as targetstr
targets = ["101010"]

# generate the simulation
video_name = "videos/grover_animation.mp4"
generate_animation(circuit, file_name=video_name, animation_steps=animation_steps, plot_titles=animation_titles, target_states=targets)
Video(video_name)

Generating frames for step: 8/8
Video saved in videos/grover_animation.mp4


### An even more complex example: Grover for RSA animated

In [this notebook](https://github.com/qperfect-io/QleoNotebooks/blob/main/usecases/cybersecurity/grover.ipynb) we also describe how to implement grover to solve an RSA factorization problem.
Using the same exact logic and by tweaking a few parameters our animation generator can scale perfectly and can offer great support for explanations of the algorithm. 

### Implementing Grover for RSA

We start by defining some usefull functions for the oracle

In [9]:
def PhiMultiply(X, Y, Z):
    """
    Acts on three registers to perform the transformation 
    |x,y,z> -> |x,y,z+x*y> in Fourier space
    """
    circ = Circuit()
    nx, ny, nz = len(X), len(Y), len(Z)
    
    # multiplication x*y 
    for j in range(nx):
        for i in range(ny):
            for k in range(i+j, nz):
                angle = 2*np.pi / 2.0**(k - j - i + 1)
                if angle % (2*np.pi) != 0:  # do nothing if angle is multiple of 2pi
                    ctr1, ctr2, target = X[j], Y[i], Z[k]
                    circ.push(Control(2,GateP(angle)), ctr1, ctr2, target)
    return circ

def Multiply(X, Y, Z):
    """
    Performs the operation |x,y,z> -> |x,y,z+xy>
    """
    circ = Circuit()
    nx, ny, nz = len(X), len(Y), len(Z)

    circ.push(QFT(nz), *Z)
    circ.append(PhiMultiply(X, Y, Z))
    circ.push(inverse(QFT(nz)), *Z)
    
    return circ

We now define Grover and update the animation parameters after each iteration

In [10]:
def factorize(N: int, K: int, nx: int, ny: int):
    # prepare registers
    nz = nx + ny + 1
    X = list(range(0, nx))
    Y = list(range(nx, nx + ny))
    Z = list(range(nx + ny, nx + ny + nz))

    
    Nj = [int(b) for b in bin(N)[2:][::-1]]  # binary digits of N, least significant first

    print(Nj)

    # initialize
    circ = Circuit()
    
    circ.push(GateH(), X + Y)
    for j, b in enumerate(Nj):
        if b == 1:
            circ.push(GateX(), Z[j])

    #### For stepping animation
    global animation_steps
    global animation_titles
    
    animation_steps.append(len(circ) - 1)# !!!!!!!!!!!!!!! First step for the animation for the initial setup state  
    animation_titles.append(f"X, Y uniform state and N encoding")
    
    # Grover loop
    for k in range(K):
        # Oracle
        #circ.push(PolynomialOracle(nx+ny, nz, 1,0,0,0), *(X+Y+Z))
        circ.append(inverse(Multiply(X, Y, Z)))
        circ.push(GateX(), Z)
        circ.push(Control(nz-1, GateZ()), *Z)
        circ.push(GateX(), Z)
        circ.append(Multiply(X, Y, Z))

        # Diffusion
        circ.push(Diffusion(nx+ny), *(X+Y))


        ### For stepping animatiom
        animation_steps.append(len(circ) - 1) # !!!!!!!!!!!!!!! New step for the animation after each grover iteration
        animation_titles.append(f"Grover iteration {k + 1}")
        
    circ.push(Measure(),X,X)
    return circ

In [11]:
# Here are the parameters used to generate the circuit
N = 143 # 13 * 11

nx, ny = 4,4
iterations = 8

For the clarity of the animation we are going to modify slightly how the y axis looks.
In this circuit:
 The first nx qubits encode the value X
 The following ny qubits emcode the value Y
 All the following qubits encode the value Z

We are going to provide the animation generator a way to convert a bit string to the interpretation X = x, Y = y, Z = z with the following function:

In [12]:
def bitstr_to_xy(bitstr):
    """converts a bit string to some other string interpretation

    Args:
        bitstr (str): a string containing 0s and 1s

    Returns:
        str: A new string used in the y axis of the animation
    """
    return f"X:{int(bitstr[:nx], 2)} Y:{int(bitstr[nx:(nx+ny)], 2)} N:{int(bitstr[(nx+ny):][::-1], 2)}\n" + bitstr
        # bitstr[:nx] + " " + bitstr[nx:(nx+ny)] + " " + bitstr[(nx+ny):]

In [13]:
# We reinitialize the animations steps 
animation_steps = []
animation_titles = []

# Generates the circuit and fills the animation steps
circuit = factorize(N, iterations, nx, ny)

[1, 1, 1, 1, 0, 0, 0, 1]


We can now generate the animation:

In [14]:
# generate circuits and execute
video_name = "videos/grover_rsa_animation.mp4"
generate_animation(circuit, file_name=video_name, animation_steps=animation_steps, plot_titles=animation_titles, round_cut=2, yaxis_filter_func=bitstr_to_xy)
# Detail: we've set round_cut = 2 to help the animation to group together bars of sligthly different values
# since round_cut = 2 the animation was able to group together bars of values 0.01 and 0.09 (it only looks for two digits)

Video(video_name)

Generating frames for step: 9/9
Video saved in videos/grover_rsa_animation.mp4


If you already know the states you want to keep track of, it is also possible to focus on them in the animation:

In [15]:
target1 = format(11, f'0{nx}b') + format(13, f'0{ny}b') + format(N, f'0{nx+ny+1}b')[::-1]
target2 = format(13, f'0{nx}b') + format(11, f'0{ny}b') + format(N, f'0{nx+ny+1}b')[::-1]
targets = [target1, target2]

video_name = "videos/grover_rsa_targeted_state_animation.mp4"
generate_animation(circuit, fps=24, file_name=video_name, animation_steps=animation_steps, plot_titles=animation_titles, target_states=targets, yaxis_filter_func=bitstr_to_xy)

Video(video_name)

Generating frames for step: 9/9
Video saved in videos/grover_rsa_targeted_state_animation.mp4


In [16]:
# For more help on generate_animation:
help(generate_animation)

Help on function generate_animation in module quantanium.visualization.animation:

generate_animation(circuit, fps=12, seconds_per_step=2, file_name='state_evolution.mp4', figsize=(15, 10), animation_steps=None, plot_titles=None, target_states=None, round_cut=5, yaxis_filter_func=<function default_filter at 0x703a91c3eca0>)
    Generates a simulation showing the evolution of the probability of getting a specific result after applying each gate.
        It is ill advised to generate a simulation of more than 10 qubits, if you decide to do it nonetheless please reduce the number of fps.

    Args:
        circuit (Circuit): The Circuit to simulate, each instruction will be applied one by one on the state.
        fps (int, optional): The number of frame per second to generate, the higher the smoother the animation will be. Defaults to 12.
        seconds_per_step (int, optional): Indicates how many seconds it will take for the animation to show the evolution of applying one instruction o